# import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# get trip

## function

In [32]:
from geopy.distance import great_circle as distance
from geopy.point import Point as Point
from math import sin, cos, atan2, sqrt, degrees, radians, pi, inf


def append_trip_list(trip_list, p_list, trip_id):
  list_tuple = [(point[0], trip_id) for point in p_list]
  return trip_list + list_tuple

def duration_min_list(list_points):
  try:
    # index 3 is timestamp
    return (list_points[-1][3] - list_points[0][3]).seconds / 60
  except:
    return 0

def distance_km(location1, location2):
    lat1, lon1 = location1
    lat2, lon2 = location2
    radius = 6371 # km

    dlat = radians(lat2-lat1)
    dlon = radians(lon2-lon1)
    a = sin(dlat/2) * sin(dlat/2) + cos(radians(lat1)) \
        * cos(radians(lat2)) * sin(dlon/2) * sin(dlon/2)
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    dist = radius * c

    # print('dist:'+str(dist))
    return dist

def is_trip_valid(trip_list):
  # check frequency ?
  # check number of point
  if len(trip_list) > 20 and distance_km((trip_list[0][1], trip_list[0][2]), (trip_list[-1][1], trip_list[-1][2])) > 1: # 1 km
    return True
  return False

def midpoint(a, b):
    a_lat, a_lon = radians(a.latitude), radians(a.longitude)
    b_lat, b_lon = radians(b.latitude), radians(b.longitude)
    delta_lon = b_lon - a_lon
    B_x = cos(b_lat) * cos(delta_lon)
    B_midpointy = cos(b_lat) * sin(delta_lon)
    mid_lat = atan2(
        sin(a_lat) + sin(b_lat),
        sqrt(((cos(a_lat) + B_x) ** 2 + B_y ** 2))
    )
    mid_lon = a_lon + atan2(B_y, cos(a_lat) + B_x)
    # Normalise
    mid_lon = (mid_lon + 3 * pi) % (2 * pi) - pi
    return Point(latitude=degrees(mid_lat), longitude=degrees(mid_lon))

def get_line_midpoint(line):
    a = Point(line[0])
    b = Point(line[1])
    return midpoint(a,b)


def calculate_dist_to_line(line_a_lat, line_a_lng, line_b_lat, line_b_lng, point_object):
    a = Point(latitude=line_a_lat, longitude=line_a_lng)
    b = Point(latitude=line_b_lat, longitude=line_b_lng)
    dist = distance(midpoint(a, b), point_object)
    return dist


def get_min_distance_to_arr(arr_coords, point_object, unit='m'):
    min_dist = inf
    
    line=[]
    for i, _ in enumerate(arr_coords):
        if i + 1 < len(arr_coords):
            dist = calculate_dist_to_line(
                line_a_lat=arr_coords[i][0],
                line_a_lng=arr_coords[i][1],
                line_b_lat=arr_coords[i + 1][0],
                line_b_lng=arr_coords[i + 1][1],
                point_object=point_object
            )
            if dist < min_dist:
                min_dist = dist
                line = [(arr_coords[i][0], arr_coords[i][1]), (arr_coords[i + 1][0], arr_coords[i + 1][1])]
        else:
            dist = calculate_dist_to_line(
                line_a_lat=arr_coords[i][0],
                line_a_lng=arr_coords[i][1],
                line_b_lat=arr_coords[0][0],
                line_b_lng=arr_coords[0][1],
                point_object=point_object
            )
            if dist < min_dist:
                min_dist = dist
                line = [(arr_coords[i][0], arr_coords[i][1]), (arr_coords[0][0], arr_coords[0][1])]
    if unit == 'm':
        return min_dist.m, line
    elif unit == 'km':
        return min_dist.km, line
    else:
        return min_dist, line

def getDistanceFromLatLonInKm(lat1,lon1,lat2,lon2) :
    R = 6371 
    dLat = deg2rad(lat2-lat1)
    dLon = deg2rad(lon2-lon1)
    a = Math.sin(dLat/2) * Math.sin(dLat/2) + Math.cos(deg2rad(lat1)) * Math.cos(deg2rad(lat2)) * Math.sin(dLon/2) * Math.sin(dLon/2)
    c = 2 * Math.atan2(Math.sqrt(a), Math.sqrt(1-a));
    d = R * c # Distance in km
    return d

def deg2rad(deg):
    return deg * (Math.PI/180)

## operate

In [33]:
!ls '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/gps_log_NE_2019-04'

2019-04-01_00.csv  2019-04-08_12.csv  2019-04-16_00.csv  2019-04-23_12.csv
2019-04-01_03.csv  2019-04-08_15.csv  2019-04-16_03.csv  2019-04-23_15.csv
2019-04-01_06.csv  2019-04-08_18.csv  2019-04-16_06.csv  2019-04-23_18.csv
2019-04-01_09.csv  2019-04-08_21.csv  2019-04-16_09.csv  2019-04-23_21.csv
2019-04-01_12.csv  2019-04-09_00.csv  2019-04-16_12.csv  2019-04-24_00.csv
2019-04-01_15.csv  2019-04-09_03.csv  2019-04-16_15.csv  2019-04-24_03.csv
2019-04-01_18.csv  2019-04-09_06.csv  2019-04-16_18.csv  2019-04-24_06.csv
2019-04-01_21.csv  2019-04-09_09.csv  2019-04-16_21.csv  2019-04-24_09.csv
2019-04-02_00.csv  2019-04-09_12.csv  2019-04-17_00.csv  2019-04-24_12.csv
2019-04-02_03.csv  2019-04-09_15.csv  2019-04-17_03.csv  2019-04-24_15.csv
2019-04-02_06.csv  2019-04-09_18.csv  2019-04-17_06.csv  2019-04-24_18.csv
2019-04-02_09.csv  2019-04-09_21.csv  2019-04-17_09.csv  2019-04-24_21.csv
2019-04-02_12.csv  2019-04-10_00.csv  2019-04-17_12.csv  2019-04-25_00.csv
2019-04-02_15.csv  2019-0

In [34]:
data_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/gps_log_NE_2019-04'
to_csv_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/trips_and_stops_NE_2019-04'

In [37]:
import glob
import os
from os.path import exists
import re

import time
start = time.time()

# days = range(1,31)
month = 4 

days = [9]


for day in days:

  export_filename = to_csv_path + f'/trips_2019_{month}_{day}.csv'

  #### check file exists
  if os.path.exists(export_filename):
    print(f'trips_2019_{month}_{day}.csv already taken!!')
    continue
  else:
    with open(export_filename, 'w') as fp:
      pass
    if os.path.exists(export_filename):
      print(f'trips_2019_{month}_{day}.csv creating...')
      pass

  #### read gps_log file
  filenames = glob.glob(data_path+f'/2019-*.csv')
  filenames = [f for f in filenames if re.search(re.compile('/2019-0?('+ str(month) +')-0?(' + str(day) + ')_[0-2][0-9]\.csv$'), f)]
  filenames = sorted(filenames)

  dfs = [pd.read_csv(filename, parse_dates=['time_stamp']).sort_values(by='time_stamp',ignore_index=True) for filename in filenames]
  df = pd.concat(dfs, ignore_index=True)
  df = df.drop_duplicates()
  df = df.drop_duplicates(subset=['time_stamp', 'unit_id'], keep="last")
  df = df[df.unit_id.isin(pd.unique(df[df.speed > 0].unit_id))]

  veh_count = df['unit_id'].value_counts()

  #### get trip algorithm
  ordered_vid = veh_count[veh_count >= 100].index
  number_of_veh = len(ordered_vid)

  STOP_DISTANCE = 0.500 # km
  STOP_DURATION = 10 # minute
  trips_list = []

  trip_id = -1
  stop_id = -1

  count = 1

  for vid in ordered_vid[:100]:
    veh_df = df[df.unit_id == vid].sort_values(by=['time_stamp'])
    poten_stop_points = []
    poten_trip_points = []
    center_cluster = (veh_df.iloc[0].index, veh_df.iloc[0]['lat'], veh_df.iloc[0]['lon'], veh_df.iloc[0]['time_stamp'])
                      #0            #1            #2            #3
    for point in zip(veh_df.index, veh_df['lat'], veh_df['lon'], veh_df['time_stamp']):
      if distance_km((point[1], point[2]), (center_cluster[1], center_cluster[2])) < STOP_DISTANCE:
        poten_stop_points.append(point)
      else:
        if duration_min_list(poten_stop_points) > STOP_DURATION and is_trip_valid(poten_trip_points):
          trip_id = trip_id + 1
          trips_list = append_trip_list(trips_list, poten_trip_points, trip_id)
          poten_trip_points = []
        else:
          poten_trip_points = poten_trip_points + poten_stop_points
        poten_stop_points = []
        center_cluster = point
        poten_stop_points.append(point)
    # last
    if duration_min_list(poten_stop_points) > STOP_DURATION and is_trip_valid(poten_trip_points):
      trip_id = trip_id + 1
      trips_list = append_trip_list(trips_list, poten_trip_points, trip_id)
      poten_trip_points = []

    

  # export
  seq_of_index, seq_of_trip_id = map(list,zip(*trips_list))
  trips_df = df.loc[seq_of_index]
  trips_df['trip_id'] = seq_of_trip_id

  trips_df.to_csv(export_filename, encoding='utf-8', index = False)
  print(f'trips_2019_{month}_{day}.csv DONE.')

end = time.time()
print(end - start)

trips_2019_4_9.csv creating...
trips_2019_4_9.csv DONE.
42.51500225067139
